In [81]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from io import StringIO, BytesIO
from azure.storage.blob import BlobServiceClient
import pandas as pd
from datetime import datetime

## Azure authentication
Don't forget to `az login` in cmd!

In [2]:
keyVaultName = "cvprojectkeyvault"
KVUri = f"https://{keyVaultName}.vault.azure.net"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

## Check Flight Manifest table before validation

In [7]:
account_url = "https://computervisionproject.blob.core.windows.net"
blob_service_client = BlobServiceClient(account_url, credential=credential)
container = 'kiosk'
blob_path = 'manifest.csv'
blob_client = blob_service_client.get_blob_client(container = container, blob = blob_path)

In [14]:
downloader = blob_client.download_blob(max_concurrency=1, encoding='UTF-8')
blobstring = downloader.readall()

In [15]:
df_manifest = pd.read_csv(StringIO(blobstring))
df_manifest

,Carrier,FlightNo,Class,From,To,Date,Baggage,Seat,Gate,BoardingTime,TicketNo,FirstName,LastName,DateOfBirth,Sex,NameValidation,DoBValidation,PersonValidation,BoardingPassValidation,LuggageValidation
0,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34A,G1,10:00 AM PST,34236746,Sameer,Kumar,1990-01-25,M,False,False,False,False,False
1,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34B,G1,10:00 AM PST,34236747,Radha S.,Kumar,1994-03-05,F,False,False,False,False,False
2,UA,234,Business,San Francisco,Chicago,2022-04-20,No,1A,G1,10:00 AM PST,34236748,James,Webb,1970-12-15,M,False,False,False,False,False
3,UA,234,Business,San Francisco,Chicago,2022-04-20,No,3D,G1,10:00 AM PST,34236749,Libby,Herold,1996-02-10,F,False,False,False,False,False
4,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,25B,G1,10:00 AM PST,34236750,James,Jackson,1956-10-12,M,False,False,False,False,False
5,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,20A,G1,10:00 AM PST,34236751,Avkash,Chauhan,1990-01-01,M,False,False,False,False,False
6,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,16F,G1,10:00 AM PST,34236752,Johannes,Czylwik,1989-05-09,M,False,False,False,False,False


## Functions to perform full validation for one passenger

In [108]:
validate_dob(df_manifest, "Johannes", "Czylwik")

1989-05-09 00:00:00
1989-05-09 00:00:00
Date of Birth Validation successful for passenger Johannes Czylwik


True

In [34]:
def get_id_card_infos():
    container = 'kiosk'
    blob_path = 'extracted_id_details/id_details.parquet'
    blob_client = blob_service_client.get_blob_client(container = container, blob = blob_path)
    downloader = blob_client.download_blob(max_concurrency=1, encoding='UTF-8')
    stream = BytesIO()
    downloader.readinto(stream)
    df_id = pd.read_parquet(stream, engine='pyarrow')
    return df_id

In [39]:
def get_boarding_pass_infos():
    container = 'kiosk'
    blob_path = 'extracted_boarding_pass_details/boarding_pass_details.parquet'
    blob_client = blob_service_client.get_blob_client(container = container, blob = blob_path)
    downloader = blob_client.download_blob(max_concurrency=1, encoding='UTF-8')
    stream = BytesIO()
    downloader.readinto(stream)
    df_bp = pd.read_parquet(stream, engine='pyarrow')
    return df_bp

In [57]:
def validate_name(manifest, first_name, last_name):
    # Check manifest name
    df_passenger = df_manifest[(df_manifest['FirstName']==first_name) & (df_manifest['LastName']==last_name)]
    if df_passenger.empty:
        print("Passenger " + first_name + " " + last_name + " not found in Flight Manifest.")
        return False
    # Check ID card name
    df_id = get_id_card_infos()
    df_pass_id = df_id[(df_id['first_name']==first_name) & (df_id['last_name']==last_name)]
    if df_pass_id.empty:
        print("Dear Sir/Madam,\nSome of the information on your ID card does not match the flight manifest data, so you cannot board the plane.\nPlease see a customer service representative.")
        return False
    # Check boarding pass name
    df_bp = get_boarding_pass_infos()
    df_pass_bp = df_bp[df_bp['Passenger Name']==first_name + " " + last_name]
    if df_pass_bp.empty:
        print("Dear Sir/Madam,\nSome of the information in your boarding pass does not match the flight manifest data, so you cannot board the plane.\nPlease see a customer service representative.")
        return False
    print("3-Way Person Name Validation successful for passenger " + first_name + " " + last_name)
    return True

In [107]:
def validate_dob(manifest, first_name, last_name):
    # Check date of birth on flight manifest
    dob_manifest = datetime.strptime(list(df_manifest.loc[(df_manifest['FirstName']==first_name) & (df_manifest['LastName']==last_name), 'DateOfBirth'])[0], '%Y-%m-%d')
    # Check date of birth on ID card
    df_id = get_id_card_infos()
    dob_id = list(df_id.loc[(df_id['first_name']==first_name) & (df_id['last_name']==last_name), 'dob'])[0]
    # Because of different date format on US driving license, month and day needs to be exchanged
    dob_id = datetime.strptime(dob_id.strftime('%Y-%m-%d'), '%Y-%d-%m')
    if dob_manifest!=dob_id:
        print("Dear Sir/Madam,\nSome of the information on your ID card does not match the flight manifest data, so you cannot board the plane.\nPlease see a customer service representative.")
        return False
    else:
        print("Date of Birth Validation successful for passenger " + first_name + " " + last_name)
        return True

In [ ]:
def validate_person(first_name, last_name):
    #TBD

In [16]:
def validate_passenger(manifest, first_name, last_name):
    print("Starting validation for passenger " + first_name + " " + last_name + "...")
    name_valid = validate_name(manifest, first_name, last_name)
    dob_valid = validate_dob(manifest, first_name, last_name)
    person_valid = validate_person(first_name, last_name)
    bp_valid = validate_boarding_pass(first_name, last_name)
    luggage_valid = validate_luggage(first_name, last_name)
    new_manifest = update_manifest(manifest, name_valid, dob_valid, person_valid, bp_valid, luggage_valid)
    print("response TBD....")
    return new_manifest

In [17]:
validate_passenger("Johannes", "Czylwik")

Starting validation for passenger Johannes Czylwik...
